In [4]:
import numpy as np
import pandas as pd

In [6]:
df=pd.read_csv("titanic.csv")
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [8]:
df=df.drop(columns=["PassengerId"])
df=df.drop(columns=["Name"])
df=df.drop(columns=["Ticket"])

In [10]:
df.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,34.5,0,0,7.8292,NaN,Q
1,1,3,female,47.0,1,0,7.0000,NaN,S
2,0,2,male,62.0,0,0,9.6875,NaN,Q


In [12]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age          86
SibSp         0
Parch         0
Fare          1
Cabin       327
Embarked      0
dtype: int64

In [14]:
df=df.drop(columns="Cabin")

In [16]:
df.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,34.5,0,0,7.8292,Q
1,1,3,female,47.0,1,0,7.0000,S


In [18]:
df=pd.get_dummies(df,columns=["Sex","Embarked"])
df.isnull().sum()

Survived       0
Pclass         0
Age           86
SibSp          0
Parch          0
Fare           1
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
dtype: int64

In [20]:
from sklearn.impute import SimpleImputer
si=SimpleImputer()
df["Age"]=si.fit_transform(df[["Age"]])
df["Fare"]=si.fit_transform(df[["Fare"]])
df.head(3)

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,34.5,0,0,7.8292,False,True,False,True,False
1,1,3,47.0,1,0,7.0000,True,False,False,False,True
2,0,2,62.0,0,0,9.6875,False,True,False,True,False


In [21]:
df.isnull().sum()

Survived      0
Pclass        0
Age           0
SibSp         0
Parch         0
Fare          0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [24]:
df=df.astype(int)

In [26]:
x=df.drop(columns="Survived")
y=df["Survived"]

In [34]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [36]:
df["Survived"].value_counts()

Survived
0    266
1    152
Name: count, dtype: int64

In [38]:
from imblearn.over_sampling import RandomOverSampler
ros=RandomOverSampler(random_state=42)
x_resampled, y_resampled = ros.fit_resample(x_train, y_train)

In [42]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_new = sc.fit_transform(x_resampled)   # fit + transform
x_test_new  = sc.transform(x_test)            # only transform


In [44]:
from sklearn.naive_bayes import GaussianNB
gaussianclassifier=GaussianNB()
gaussianclassifier.fit(x_train_new,y_resampled)

GaussianNB()

In [56]:
y_resampled.value_counts()

Survived
0    216
1    216
Name: count, dtype: int64

In [46]:
y_pred = gaussianclassifier.predict(x_test_new)

In [48]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[50,  0],
       [ 0, 34]], dtype=int64)

In [50]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

1.0

In [52]:
from sklearn.metrics import f1_score
f1_score(y_test,y_pred)

1.0

In [58]:
param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6]
}

In [60]:
from sklearn.model_selection import GridSearchCV
gridsearch = GridSearchCV(gaussianclassifier,param_grid = param_grid, cv = 9, verbose=True,n_jobs=-1)
gridsearch

GridSearchCV(cv=9, estimator=GaussianNB(), n_jobs=-1,
             param_grid={'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06]},
             verbose=True)

In [62]:
bestgridsearch = gridsearch.fit(x_resampled,y_resampled)
bestgridsearch.best_estimator_

Fitting 9 folds for each of 4 candidates, totalling 36 fits


GaussianNB()

In [66]:
print(f'Accuracy : {bestgridsearch.score(x,y):.3f}')

Accuracy : 1.000
